<a href="https://colab.research.google.com/github/hjn14133/Machine-Learning/blob/main/Random_Forest_Temperature_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd

# Load the data
features = pd.read_csv('temps.csv')
features.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


In [22]:
# Identify anomalies and missing data
print('The shape of our feature is: ', features.shape)

The shape of our feature is:  (348, 12)


In [5]:
# Descriptive statistics for each column
features.describe()

,year,month,day,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103,57.238506,62.373563,59.772989,60.034483
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146,10.605746,10.549381,10.705256,15.626179
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000,41.000000,46.000000,44.000000,28.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000,48.000000,53.000000,50.000000,47.750000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000,56.000000,61.000000,58.000000,60.000000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000,66.000000,72.000000,69.000000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000,77.000000,82.000000,79.000000,95.000000


In [ ]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)

features.iloc[:, 5:].head()


In [24]:
features = features.drop(['forecast_noaa', 'forecast_acc', 'forecast_under'] , 
              axis = 1)

print(features.shape)

(348, 15)


In [ ]:
features.iloc[:, 5:].head()

In [25]:
import numpy as np

# Labels are the values we want to predict
labels = np.array(features['actual'])

features= features.drop('actual', axis = 1)

# saving feature names for later use
features_list = list(features.columns)

# Convert pandas dataframe to numpy array
features = np.array(features)

In [26]:
# Training and Testing Sets

from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                            labels,
                                                                            test_size = 0.25,
                                                                            random_state = 42)

In [28]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (261, 14)
Training Labels Shape: (261,)
Testing Features Shape: (87, 14)
Testing Labels Shape: (87,)


In [29]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:,
                               features_list.index('average')]

# baseline errors and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)

print('average baseline error: ', round(np.mean(baseline_errors), 2))

average baseline error:  5.06


In [30]:
# Traing the model with random forest model
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [34]:
# Make predictions on the test set
predictions = rf.predict(test_features)

# Calculate the absolute error
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error of Random Forest Prediction: ', 
      round(np.mean(errors), 2), 'degrees')

Mean Absolute Error of Random Forest Prediction:  3.83 degrees


In [35]:
# Determine Performance Metrics
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

print('Accuracy: ', round(accuracy, 2), '%')

Accuracy:  93.98 %
